In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline

In [2]:
path = "./"
train = pd.read_csv(path + "train.csv",true_values='t',false_values='f')
test = pd.read_csv(path + "test.csv",true_values='t',false_values='f')

## Checking & Modifying Column Data Types

In [3]:
train['first_review'] = pd.to_datetime(train['first_review'])
train['host_since'] = pd.to_datetime(train['host_since'])
train['last_review'] = pd.to_datetime(train['last_review'])
train.dtypes

id                                 int64
log_price                        float64
property_type                     object
room_type                         object
amenities                         object
accommodates                       int64
bathrooms                        float64
bed_type                          object
cancellation_policy               object
cleaning_fee                        bool
city                              object
description                       object
first_review              datetime64[ns]
host_has_profile_pic              object
host_identity_verified            object
host_response_rate                object
host_since                datetime64[ns]
instant_bookable                    bool
last_review               datetime64[ns]
latitude                         float64
longitude                        float64
name                              object
neighbourhood                     object
number_of_reviews                  int64
review_scores_ra

## Calculating NaN %

In [6]:
train_nan = (train.isnull().sum() / train.shape[0]) * 100
train_nan = train_nan[train_nan>0]
train_nan

bathrooms                  0.269865
first_review              21.405729
host_has_profile_pic       0.253674
host_identity_verified     0.253674
host_response_rate        24.691341
host_since                 0.253674
last_review               21.355804
neighbourhood              9.272578
review_scores_rating      22.563452
thumbnail_url             11.086074
zipcode                    1.303450
bedrooms                   0.122789
beds                       0.176762
dtype: float64

## One-hot-encode categorical variables

In [26]:
train['dataset'] = "train"
test['dataset'] = "test"
data = pd.concat([train,test], axis = 0)

# Create amenities columns
amenitie_options = []

for a in data.amenities:
    a = a[1:-1]
    
    if len(a)>0 and a[0]=='"':
        a = a[1:]
        
    if len(a)>0 and a[-1]=='"':
        a = a[:-1]
    
    for x in a.split('","'):
        for y in x.split(',"'):
            for z in y.split('",'):
                if ', ' in z:
                    # Don't split
                    z = z.replace(',','')
                    if len(z)>0 and 'translation missing' not in z and z not in amenitie_options:
                        amenitie_options.append(str(z))
                else:
                    for a in z.split(','):
                        if len(a)>0 and 'translation missing' not in a and a not in amenitie_options:
                            amenitie_options.append(str(a))
amenitie_options.sort()
amenitie_options.remove('Firm matress')
for row in amenitie_options:
    print(row)
print(len(amenitie_options))

24-hour check-in
Accessible-height bed
Accessible-height toilet
Air conditioning
Air purifier
BBQ grill
Baby bath
Baby monitor
Babysitter recommendations
Bath towel
Bathtub
Bathtub with shower chair
Beach essentials
Beachfront
Bed linens
Body soap
Breakfast
Buzzer/wireless intercom
Cable TV
Carbon monoxide detector
Cat(s)
Changing table
Children’s books and toys
Children’s dinnerware
Cleaning before checkout
Coffee maker
Cooking basics
Crib
Disabled parking spot
Dishes and silverware
Dishwasher
Dog(s)
Doorman
Doorman Entry
Dryer
EV charger
Elevator
Elevator in building
Essentials
Ethernet connection
Extra pillows and blankets
Family/kid friendly
Fire extinguisher
Fireplace guards
Firm mattress
First aid kit
Fixed grab bars for shower & toilet
Flat smooth pathway to front door
Free parking on premises
Free parking on street
Game console
Garden or backyard
Grab-rails for shower and toilet
Ground floor access
Gym
Hair dryer
Hand or paper towel
Hand soap
Handheld shower head
Hangers
Heatin

In [14]:
categorical = ['property_type','room_type','bed_type','cancellation_policy','city']
data = pd.get_dummies(data, columns = categorical)
data.columns

NameError: name 'data' is not defined

In [15]:
# Select only numeric data and impute missing values as 0
numerics = ['uint8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
train_x = data[data.dataset == "train"] \
    .select_dtypes(include=numerics) \
    .drop("log_price", axis = 1) \
    .fillna(0) \
    .values

test_x = data[data.dataset == "test"] \
    .select_dtypes(include=numerics) \
    .drop("log_price", axis = 1) \
    .fillna(0) \
    .values

train_y = data[data.dataset == "train"].log_price.values

NameError: name 'data' is not defined

In [54]:
# Train a Random Forest model with cross-validation

from sklearn.model_selection import KFold
cv_groups = KFold(n_splits=5)
regr = RandomForestRegressor(random_state = 0, n_estimators = 10)

for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)

RMSE for current split: 0.422514098591
RMSE for current split: 0.419582744545
RMSE for current split: 0.413761925371
RMSE for current split: 0.421351916657
RMSE for current split: 0.424273559012


In [14]:
# Create submission file
regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)

submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','log_price'])
submission.to_csv("sample_submission.csv", index = False)